<a href="https://colab.research.google.com/github/antoniomuso/speech2face/blob/master/NewDecoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
vox1_path = "/content/drive/My Drive/Speech2Face/vox1_dataset/"
img = vox1_path+"unzippedFaces"


In [ ]:
# !wget http://www.robots.ox.ac.uk/~vgg/research/CMBiometrics/data/zippedFaces.tar.gz 
!tar zxvf zippedFaces.tar.gz -C "/content/drive/My Drive/Speech2Face/vox1_dataset/"

Streaming output truncated to the last 5000 lines.
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0000575.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0006650.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0002175.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0023950.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0014200.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0006475.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0016125.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0007275.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0007050.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0011475.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0011125.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0002400.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0003150.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0011275.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0021200.jpg
unzippedFaces/Pablo_Schreiber/1.6/cevM52fwmh8/0003200.jpg
unzippedFaces/Pablo_S

In [ ]:
# !curl --user voxceleb1912:0s42xuw6 -o "/content/drive/My Drive/Speech2Face/ff/vox.zip" http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox1_test_wav.zip

In [2]:
import glob

import numpy as np
import sys

from PIL import Image
from torch.utils.data import Dataset
import  torchvision.transforms as transforms


def load_metadata(data_file_path):
    data_list = []
    with open(data_file_path, "r") as f:
        for line in f:
            data_list.append(line.rstrip("\n").split(","))
    return data_list


class _Normalize_Tensor(object):
    def __init__(self, color_mean, color_std):
        self.color_mean = color_mean
        self.color_std = color_std

    def __call__(self, img):
        # Convert image to Tensor
        img = transforms.functional.to_tensor(img)

        # Normalize image by the parameter of pre-trained face-encoder
        img = transforms.functional.normalize(
            img, self.color_mean, self.color_std)

        return img

class EmbedImagePairs(Dataset):
    def __init__(self, data_list_path, face_encoder, mean, std, device, size=64):
        super().__init__()

        #data_list = load_metadata(data_list_path)[:-100]
        data_list = load_metadata(data_list_path)
        print("Load {} data".format(len(data_list)))
        self.data_list = [face_path[0] for face_path in data_list]

        self.face_encoder = face_encoder

        self.color_mean = mean
        self.color_std = std
        
        self.device = device
        self.size = size

        # Transform input for face-encoder
        self.transform_fe = transforms.Compose([
            transforms.Resize((224, 224)),
            _Normalize_Tensor(self.color_mean, self.color_std)
        ])

        self.transform_fd = transforms.Compose([
            transforms.Resize((self.size, self.size)),
            transforms.ToTensor()
        ])
    
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        # Load face-image
        image = Image.open(self.data_list[idx])

        # convert face-images into 4096-dim vectors
        self.face_encoder.eval()
        face_vectors = self.face_encoder(
            self.transform_fe(image).unsqueeze_(0).to(self.device))

        image_ = self.transform_fd(image)

        return face_vectors, image_

In [3]:
import torch
import torch.nn as nn


class Vgg_face_dag(nn.Module):

    def __init__(self):
        super(Vgg_face_dag, self).__init__()
        self.meta = {'mean': [129.186279296875, 104.76238250732422, 93.59396362304688],
                     'std': [1, 1, 1],
                     'imageSize': [224, 224, 3]}
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.fc6 = nn.Linear(in_features=25088, out_features=4096, bias=True)
        self.relu6 = nn.ReLU(inplace=True)
        self.dropout6 = nn.Dropout(p=0.5)
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.relu7 = nn.ReLU(inplace=True)
        self.dropout7 = nn.Dropout(p=0.5)
        self.fc8 = nn.Linear(in_features=4096, out_features=2622, bias=True)

    def forward(self, x0, is_fc8=False):
        x1 = self.conv1_1(x0)
        x2 = self.relu1_1(x1)
        x3 = self.conv1_2(x2)
        x4 = self.relu1_2(x3)
        x5 = self.pool1(x4)
        x6 = self.conv2_1(x5)
        x7 = self.relu2_1(x6)
        x8 = self.conv2_2(x7)
        x9 = self.relu2_2(x8)
        x10 = self.pool2(x9)
        x11 = self.conv3_1(x10)
        x12 = self.relu3_1(x11)
        x13 = self.conv3_2(x12)
        x14 = self.relu3_2(x13)
        x15 = self.conv3_3(x14)
        x16 = self.relu3_3(x15)
        x17 = self.pool3(x16)
        x18 = self.conv4_1(x17)
        x19 = self.relu4_1(x18)
        x20 = self.conv4_2(x19)
        x21 = self.relu4_2(x20)
        x22 = self.conv4_3(x21)
        x23 = self.relu4_3(x22)
        x24 = self.pool4(x23)
        x25 = self.conv5_1(x24)
        x26 = self.relu5_1(x25)
        x27 = self.conv5_2(x26)
        x28 = self.relu5_2(x27)
        x29 = self.conv5_3(x28)
        x30 = self.relu5_3(x29)
        x31_preflatten = self.pool5(x30)
        x31 = x31_preflatten.view(x31_preflatten.size(0), -1)
        x32 = self.fc6(x31)
        x33 = self.relu6(x32)
        x34 = self.dropout6(x33)
        x35 = self.fc7(x34)
        x36 = self.relu7(x35)
        x37 = self.dropout7(x36)
        if is_fc8:
            x38 = self.fc8(x37)
        else:
            x38 = x37
        return x38


def vgg_face_dag(weights_path=None, **kwargs):
    """
    load imported model instance
    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = Vgg_face_dag()
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model

In [4]:
import torch
import torch.nn.functional as F
from torch import nn

class Generator(nn.Module):
    def __init__(self, n_hidden, bottom_width=4, channels=512):
        super().__init__()
        self.channels = channels
        self.bottom_width = bottom_width

        self.linear = nn.Linear(n_hidden, bottom_width*bottom_width*channels)
        self.dconv1 = nn.ConvTranspose2d(channels, channels // 2, 4, 2, 1)
        self.dconv2 = nn.ConvTranspose2d(channels // 2, channels // 4, 4, 2, 1)
        self.dconv3 = nn.ConvTranspose2d(channels // 4, channels // 8, 4, 2, 1)
        self.dconv4 = nn.ConvTranspose2d(channels // 8, 3, 4, 2, 1)

        self.bn0 = nn.BatchNorm1d(bottom_width*bottom_width*channels)
        self.bn1 = nn.BatchNorm2d(channels // 2)
        self.bn2 = nn.BatchNorm2d(channels // 4)
        self.bn3 = nn.BatchNorm2d(channels // 8)

    def forward(self, x):
        x = F.relu(self.bn0(self.linear(x))).view(-1, self.channels, self.bottom_width, self.bottom_width)
        x = F.relu(self.bn1(self.dconv1(x)))
        x = F.relu(self.bn2(self.dconv2(x)))
        x = F.relu(self.bn3(self.dconv3(x)))

        x = torch.sigmoid(self.dconv4(x))

        return x

In [ ]:
import argparse
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from glob import glob
import os
from os.path import join
from os import listdir
from os.path import isdir, isfile
import subprocess
import sys
import warnings

import cv2
import face_recognition
import numpy as np 
from tqdm import tqdm

def make_list(args):
    metadata_list = []
    image_path_list = glob(join(args.data_dir, "*.jpg"))
    for image_path in tqdm(image_path_list):
        name, ext = os.path.basename(image_path).split(".")
        trimmed_image_path = join(args.data_dir, "trimmed_" + name + ".jpg")

        # if the trimmed image already exists
        if isfile(trimmed_image_path):
            image_path_list.append(trimmed_image_path)
            continue

        try:
            face = face_recognition.load_image_file(image_path)
            image = cv2.imread(image_path)
            
            face_locations = face_recognition.face_locations(face, model="cnn")[0]

            face_cut = image[face_locations[0] : face_locations[2],
                            face_locations[3] : face_locations[1]]

            cv2.imwrite(trimmed_image_path, face_cut)
            metadata_list.append(trimmed_image_path)
        except:
            continue

    return metadata_list

def _save_metadata(data_list, args):
    saved_path = join(args.base_dir, args.file_name)
    with open(saved_path, "w", encoding="utf-8") as f:
        for image_path in data_list:
            f.write(image_path + "," + "_.wav" + "\n")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--base_dir', default=os.path.expanduser('~/Speech2Face'))
    parser.add_argument('--data_dir', default=os.path.expanduser('~/Speech2Face'))
    parser.add_argument('--file_name', default='train.txt')
    args = parser.parse_args()

    data_list = make_list(args)
    _save_metadata(data_list, args)

In [5]:
class Compose(object):
    """引数transformに格納された変形を順番に実行するクラス
    """
    def __init__(self, transforms):
        self.transforms = transforms
        
    def __call__(self, x):
        for t in self.transforms:
            x = t(x)
        return x
    
class Resize(object):
    def __init__(self, input_size):
        self.input_size = input_size
        
    def __call__(self, img):
        img = img.resize((self.input_size, self.input_size),
                                Image.BICUBIC)
        return img
    
class Normalize_Tensor(object):
    def __init__(self, color_mean, color_std):
        self.color_mean = color_mean
        self.color_std = color_std

    def __call__(self, img):

        # PIL画像をTensorに。大きさは最大1に規格化される
        img = transforms.functional.to_tensor(img)

        # 色情報の標準化
        img = transforms.functional.normalize(
            img, self.color_mean, self.color_std)

        return img

class ImageTransform():
    def __init__(self, input_size, color_mean, color_std):
        self.data_transform = {
            'train' : Compose([
                Resize(input_size),
                Normalize_Tensor(color_mean, color_std)
            ]),
            'val' : Compose([
                Resize(input_size),
                Normalize_Tensor(color_mean, color_std)
            ])
        }
    
    def __call_(self, phase, img):
        return self.data_transform[phase](img)

In [ ]:
import argparse
import os
from os.path import join

import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from tqdm import tqdm

from dataloader import EmbedImagePairs
from face_encoder_model import vgg_face_dag
from face_decoder_model import Generator


global_epoch = 0

def _save_model(epoch, model, optimizer, fname):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, join(output_dir_name, 'epoch_{}.pth'.format(epoch)))


def train_epoch(model, trn_dataloader, criterion, optimizer, device):
    model.train()

    running_loss = 0

    for x, y in trn_dataloader:
        x = x.squeeze().to(device)
        y = y.to(device)

        optimizer.zero_grad()

        outputs = model(x)

        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
    
    trn_loss = running_loss/len(trn_dataloader.dataset)
    
    return trn_loss


def train(model, dataloaders, criterion, optimizer, device, output_dir_name, num_epochs=100):
    trn_losses = []

    # make dir where trained model saved
    os.makedirs(output_dir_name, exist_ok=True)

    # start at zero, end at num_epochs (including)
    print("Start Training")
    for epoch in tqdm(range(global_epoch, num_epochs+1)):
        # Training phase
        trn_loss = train_epoch(model, dataloaders, criterion, optimizer, device)
        trn_losses += [trn_loss]
        print("Epoch: {}  || Loss: {}".format(epoch, trn_loss))
    
        _save_model(epoch, model, optimizer, output_dir_name)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--base_dir', default=os.path.expanduser('~/Speech2Face'))
    parser.add_argument("--input", default="train.txt")
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--num_epochs", type=int, default=150)
    parser.add_argument("--checkpoint_path", default=None)
    args = parser.parse_args()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Set Model
    model = Generator(4096).to(device)

    # Load face-encoder model
    face_encoder_path = join(args.base_dir, "models", "vgg_face_dag.pth")
    face_encoder_model = vgg_face_dag(face_encoder_path).to(device)
    color_mean, color_std = face_encoder_model.meta["mean"], face_encoder_model.meta["std"]
    color_mean = [tmp / 255.0 for tmp in color_mean]
    color_std = [tmp / 255.0 for tmp in color_std]

    # Set loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())

    # Set DataLoader
    input_path = join(args.base_dir, args.input)
    dataset = EmbedImagePairs(input_path, face_encoder_model,
                              color_mean, color_std, device)
    trn_dataloader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True)

    output_dir_name = join(args.base_dir, "models", "face_decoder")

    if args.checkpoint_path:
        checkpoint = torch.load(args.checkpoint_path)
        global_epoch = checkpoint["epoch"]
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    # Train
    train(model, trn_dataloader, criterion, optimizer, device, output_dir_name,
            num_epochs=args.num_epochs)
    print("Finish training")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from dataloader import EmbedImagePairs
from face_decoder_model import Generator

# face_detector = dlib.get_frontal_face_detector()
# landmark_detector = dlib.shape_predictor('models/dlib/shape_predictor_68_face_landmarks.dat')
# face_embedder = dlib.face_recognition_model_v1('models/dlib/dlib_face_recognition_resnet_model_v1.dat')

def train_epoch(model, trn_dataloader, criterion, optimizer, device):
    model.train()

    running_loss = 0

    for x, y in trn_dataloader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        outputs = model(x)

        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
    
    trn_loss = running_loss/len(trn_dataloader.dataset)
    
    return trn_loss

def distance_metric(pred_photo, input_emb, dlib_models, ret_emb=False):
    face_detector, landmark_detector, face_embedder = dlib_models
    
    dist = None 
    embedding = None
    
    face_boxes = face_detector(pred_photo, 1)

    if len(face_boxes) == 1:
        landmarks = landmark_detector(pred_photo, face_boxes[0])
        embedding = np.array(face_embedder.compute_face_descriptor(pred_photo, landmarks))
        
        assert embedding.shape == input_emb.shape; "shape of true embedding and predicted embedding not same"
        
        dist = np.linalg.norm(np.array(embedding) - np.array(input_emb))
    
    if ret_emb:
        return dist, embedding
    else:
        return dist

def val_epoch(model, val_dataloader, criterion, dlib_models, device):
    model.eval() 
    dists = []

    running_loss = 0

    for x, y in val_dataloader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)

        loss = criterion(outputs, y)
        running_loss += loss.item() * x.size(0)
        
        x = x.cpu().numpy()
        photos = outputs.detach().permute(0, 2, 3, 1).cpu().numpy()
        photos = (photos*255).astype('uint8')
            
        for input_emb, photo in zip(x, photos):
            dist = distance_metric(photo, input_emb, dlib_models)
            if dist is not None: dists += [dist]
        
    val_loss = running_loss/len(val_dataloader.dataset)

    return val_loss, dists

def save_model(epoch, model, optimizer, fname):
    print(f'saving at epoch {epoch}')
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, f'{fname}_{epoch}.pth')


def train(model, dataloaders, criterion, optimizer, device, out_name, dlib_models=None, validate=False, validate_every=10, num_epochs=100):
    trn_losses = []
    
    if validate:
        avg_dists = []
        val_losses = []
        
        assert len(dataloaders) == 2
        assert dlib_models is not None
    
    # start at zero, end at num_epochs (including)
    for epoch in tqdm(range(1, num_epochs+1)):
        # Training phase
        trn_loss = train_epoch(model, dataloaders['train'], criterion, optimizer, device)
        trn_losses += [trn_loss]

        # Validation Phase
        if epoch % validate_every == 0:
            save_model(epoch, model, optimizer, out_name)

            if validate:
                val_loss, dists = val_epoch(model, dataloaders['val'], criterion, dlib_models, device)
                val_losses += [val_loss]

                avg_dist = np.mean(dists)
                avg_dists += [avg_dist]

                np.save(out_name + str(epoch) + "_dists.npy", dists)

                print("Epoch: ", epoch, "Train Loss:", trn_loss, "Val Loss:", val_loss, "Average Distance:", avg_dist)
    
    save_model(epoch, model, optimizer, out_name + "Final")
    
    np.save(out_name + "trn_losses.npy", trn_losses)
    
    if validate: 
        np.save(out_name + "val_losses.npy", val_losses)
        np.save(out_name + "avg_dists.npy", avg_dists)

def test(model, test_dataloader, dlib_models, device, raw_dists=False):
    model.eval() 
    dists = []
    
    if raw_dists:
        true_vecs = []
        synth_vecs = []
        
    for x, y in test_dataloader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)
        
        x = x.cpu().numpy()
        photos = outputs.detach().permute(0, 2, 3, 1).cpu().numpy()
        photos = (photos*255).astype('uint8')
            
        for input_emb, photo in zip(x, photos):
            if not raw_dists:
                dist = distance_metric(photo, input_emb, dlib_models, raw_dists)
                if dist is not None: dists += [dist]
            else:
                dist, embedding = distance_metric(photo, input_emb, dlib_models, raw_dists)
                true_vecs += [input_emb]
                synth_vecs += [embedding]
    
    if raw_dists:
        return true_vecs, synth_vecs
    else:
        return dists

def synthesize(model, vis_dataloader, device):
    model.eval()
    
    all_photos = []
    
    for x, y in vis_dataloader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)
        
        x = x.cpu().numpy()
        photos = outputs.detach().permute(0, 2, 3, 1).cpu().numpy()
        photos = (photos*255).astype('uint8')
        
        all_photos += [photos]
    
    all_photos = np.concatenate(all_photos, axis=0)
    names = vis_dataloader.dataset.embeds
    
    return all_photos, names